# Installing Libraries

In [ ]:
!pip install -q transformers datasets
!pip install peft
!pip install -U fvcore
!pip install thop
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=757223537f688eca28452fdae6cb53ed9b9b5be05c872bdef10490320348ac4f
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir DriveLM
!unzip -q drive/MyDrive/DriveLM/data.zip -d DriveLM

In [ ]:
import os
import pdb
import time
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering, \
TrainingArguments, Trainer, BertTokenizerFast, Blip2ForConditionalGeneration, Blip2Processor, AutoTokenizer
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
from peft import LoraConfig, get_peft_model, LoftQConfig
from copy import deepcopy
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import json
import asyncio
import pdb
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from fvcore.nn import FlopCountAnalysis

class SingleFrameDataset(Dataset):

    def __init__(self, input_file, processor, custom_train=True):
        with open(input_file) as f:
            self.data = json.load(f)

        # Make processors for Image-Question pairs and Answer text
        self.processor = processor

        self.custom_train = custom_train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the question and answer at the idx
        qa, img_path = self.data[idx]
        img_path = os.path.join('DriveLM', '/'.join(list(img_path.values())[0].split('\\')))
        q_text, a_text = qa['Q'], qa['A']
        new_q_text = f"Question: {q_text} Answer:"
        full_text = f"Question: {q_text} Answer: {a_text}"
        img = read_image(img_path)
        # enc = self.processor(images=img, text=new_q_text, return_tensors='pt')
        enc = self.processor(new_q_text, return_tensors='pt')
        enc.to(device)

        return enc, img

## FLOPs for LLaMA Model

In [ ]:
!mkdir ./llama-checkpoint
!python /usr/local/lib/python3.10/dist-packages/transformers/models/llama/convert_llama_weights_to_hf.py \
    --input_dir ./llama-2-7b/ --model_size 7B --output_dir ./llama-checkpoint

mkdir: cannot create directory ‘./llama-checkpoint’: File exists
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Fetching all parameters from the checkpoint at ./llama-2-7b/.
Loading the checkpoint in a Llama model.
Loading checkpoint shards:   0% 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("huggyllama/llama-7b")
model = LlamaForCausalLM.from_pretrained("huggyllama/llama-7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model
    """

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"Trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
dataset = SingleFrameDataset(os.path.join('DriveLM', 'data', 'multi_frame', 'multi_frame_train.json'), tokenizer)

In [ ]:
# model.to(device)

flops_average = []
avg_time_diff = []

for i in range(20):

    input, _ = dataset[i]
    flops = FlopCountAnalysis(model, (input['input_ids'], input['attention_mask']))
    with torch.no_grad():
      x = time.time()
      model(**input)
      avg_time_diff.append(time.time() - x)
    flops_average.append(flops.total())

sum(flops_average) / len(flops_average), sum(avg_time_diff) / len(avg_time_diff)

(268718235238.4, 0.10324854850769043)

## FLOPs for BLIP2 Model

In [ ]:
model = Blip2ForConditionalGeneration.from_pretrained('Salesforce/blip2-opt-2.7b')
processor = Blip2Processor.from_pretrained('Salesforce/blip2-opt-2.7b')
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [ ]:
dataset = SingleFrameDataset(os.path.join('DriveLM', 'data', 'single_frame', 'single_frame_train.json'), processor)

flops_average = []
avg_time_diff = []

for i in range(20):

    input, _ = dataset[i]
    flops = FlopCountAnalysis(model, (input['pixel_values'], input['input_ids'], input['attention_mask']))
    with torch.no_grad():
      x = time.time()
      model(**input)
      avg_time_diff.append(time.time() - x)
    flops_average.append(flops.total())

sum(flops_average) / len(flops_average), sum(avg_time_diff) / len(avg_time_diff)

(439401165312.0, 0.18045687675476074)

## Flops for Vit-g/14 Model

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel

model = CLIPVisionModel.from_pretrained("laion/CLIP-ViT-g-14-laion2B-s12B-b42K")
processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-g-14-laion2B-s12B-b42K")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.47G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
dataset = SingleFrameDataset(os.path.join('DriveLM', 'data', 'single_frame', 'single_frame_train.json'), processor)

In [ ]:
model.to(device)

flops_average = []
avg_time_diff = []

for i in range(20):

    _, img = dataset[i]
    input = processor(text=None, images=img, return_tensors='pt').to(device)
    flops = FlopCountAnalysis(model, (input['pixel_values']))
    flops_average.append(flops.total())
    with torch.no_grad():
      x = time.time()
      model(**input)
      avg_time_diff.append(time.time() - x)

sum(flops_average) / len(flops_average), sum(avg_time_diff) / len(avg_time_diff)

(267176642304.0, 0.02991529703140259)

## FLOPs for CLIP Model

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel

model = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

dataset = SingleFrameDataset(os.path.join('DriveLM', 'data', 'single_frame', 'single_frame_train.json'), processor)

model.to(device)

flops_average = []
avg_time_diff = []

for i in range(20):

    _, img = dataset[i]
    input = processor(text=None, images=img, return_tensors='pt').to(device)
    flops = FlopCountAnalysis(model, (input['pixel_values']))
    flops_average.append(flops.total())

    with torch.no_grad():
      x = time.time()
      model(**input)
      avg_time_diff.append(time.time() - x)

sum(flops_average) / len(flops_average), sum(avg_time_diff) / len(avg_time_diff)

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel

model = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14")
print_trainable_parameters(model)

## FLOPs for EM-VLM4AD

In [ ]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision import transforms
import torch
import json
import os
import os
from collections import namedtuple
from tqdm import tqdm as progress_bar
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import LoraConfig, get_peft_model, LoftQConfig
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import vit_b_32
import json
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MultiFrameDataset(Dataset):

    def __init__(self, input_file, tokenizer, transform=None):
        with open(input_file) as f:
            self.data = json.load(f)

        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the question and answer at the idx
        qa, img_path = self.data[idx]
        img_paths = list(img_path.values())

        q_text, a_text = qa['Q'], qa['A']
        q_text = f"Question: {q_text} Answer:"

        # Concatenate images into a single tensor
        imgs = [self.transform(read_image(p).float()).to(device) for p in img_paths]
        imgs = torch.stack(imgs, dim=0)

        encodings = self.tokenizer(q_text, padding=True, return_tensors="pt").input_ids.to(device)
        labels = self.tokenizer(a_text, padding=True, return_tensors='pt').input_ids.to(device)

        return q_text, imgs, labels

In [ ]:
class DriveVLMT5(nn.Module):

    def __init__(self, config):

        super().__init__()

        # Make tokenizer and text model
        if config.lm == 'T5-Base':
            self.model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-base')
        else:
            self.model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-large')

            # For quantization
            loftq_config = LoftQConfig(loftq_bits=8)
            # Create LoRA model
            lora_config = LoraConfig(
                r=config.lora_dim,
                lora_alpha=config.lora_alpha,
                loftq_config=loftq_config,
                lora_dropout=config.lora_dropout,
                bias='none',
                target_modules=['q', 'v']
            )
            self.model = get_peft_model(self.model, lora_config)

        hidden_size = self.model.config.d_model

        print('Trainable Parameters for LM model:')
        print_trainable_parameters(self.model)

        # Create instance for multi-view processor
        self.mvp = self.MultiViewProcessor(config.gpa_hidden_size, hidden_size, config.lm, freeze=True)

    class MultiViewProcessor(nn.Module):

        def __init__(self, gpa_hidden_size, hidden_size, lm, freeze=False):

            super().__init__()

            # Use ViT for image embeddings
            self.img_model = vit_b_32(weights='DEFAULT')
            self.lm = lm

            # Modal embedding to distinguish between image and text
            self.modal_embeddings = nn.Embedding(2, hidden_size)
            self.modal_embeddings.weight.data.normal_(mean=0.0, std=0.02)

            # If we are freezing the CLIP embeddings
            if freeze:
                for param in self.img_model.parameters():
                    param.requires_grad = False

            # Set matrices based on MIVC paper
            self.w = nn.Linear(in_features=gpa_hidden_size, out_features=1)
            self.Z = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Tanh()
            )
            self.G = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Sigmoid()
            )

            if self.lm != 'T5-Base':
              self.img_projection_layer = nn.Linear(in_features=VIT_HIDDEN_STATE, out_features=hidden_size)


        def gpa(self, img_embeddings):

            """"
            Calculates the gated-pooling attention score for the image embeddings
            :param img_embeddings: (6x768) dimensional
            :return single embedding of size (768,)
            """

            # Get weights for gated pooling attention
            gpa_weights = torch.softmax(self.w(self.Z(img_embeddings) * self.G(img_embeddings)), dim=0  )

            # Take a linear combination of all the image embeddings
            fused_embeddings = torch.sum(gpa_weights * img_embeddings, dim=0)

            return fused_embeddings

        def get_img_embedding(self, imgs):

            N = imgs.shape[0]

            # Process into patches (N x 6 x 49 x H)
            merged_embedding = torch.stack([self.img_model._process_input(img) for img in imgs], dim=0)

            # Concatenate the batch class tokens -> (N, 6, 50, H)
            batch_class_tokens = self.img_model.class_token.expand(merged_embedding.shape[1], -1, -1).repeat(N, 1, 1, 1)
            merged_embedding = torch.cat([batch_class_tokens, merged_embedding], dim=2)

            # Add positional embeddings and remove class token -> (N, 6, 49, H)
            merged_embedding += self.img_model.encoder.pos_embedding.repeat(N, 1, 1, 1)
            merged_embedding = merged_embedding[:, :, 1:]

            # Get merged embedding and reshape to 2D embedding -> (N, 1, 49, H)
            merged_embedding = torch.stack([self.gpa(embedding.flatten(start_dim=1)).reshape(VIT_SEQ_LENGTH,
                                            VIT_HIDDEN_STATE) for embedding in merged_embedding], dim=0)

            # Project to VL dimension -> (1, 49, H) (H is 512 for t5-small, 768 for t5-base)
            if self.lm != 'T5-Base':
              merged_embedding = self.img_projection_layer(merged_embedding)

            # Add modal type embedding to merged embedding
            merged_embedding += self.modal_embeddings(
                torch.ones((1, merged_embedding.shape[1]), dtype=torch.int, device=device))

            return merged_embedding

        def forward(self, text_enc, imgs, text_model):

            # Get the image embeddings (N x 1 x 49 x H)
            imgs_embedding = self.get_img_embedding(imgs)

            # Get the text embeddings (N x S x H)
            text_embeddings = text_model.get_input_embeddings()(text_enc)

            # Add modal embeddings to text
            text_embeddings += self.modal_embeddings(torch.zeros((1, text_embeddings.shape[1]), dtype=torch.int,
                                                                 device=device))

            # Concatenate embeddings -> (1 x S x 512)
            merged_embedding = torch.cat([text_embeddings, imgs_embedding], dim=1)

            return merged_embedding

    def forward(self, text_enc, imgs, labels=None):

        # Get the merged embeddings
        merged_embedding = self.mvp(text_enc, imgs, self.model)

        # If training include the labels
        return self.model(inputs_embeds=merged_embedding, labels=labels)

In [ ]:
Config = namedtuple('Instance', ['batch_size', 'gpa_hidden_size',
                                 'lora', 'lora_dim', 'lora_alpha', 'lora_dropout',
                                 'model_name', 'lm', 'max_len'])

config = Config(
    batch_size = 16,
    gpa_hidden_size = 128,
    lora = False,
    lora_dim = 64,
    lora_alpha = 32,
    lora_dropout = 0.05,
    model_name = '20240301-053312',
    lm = 'T5-Base',
    max_len = 512
)

In [ ]:
def val_model(dloader):

    model.eval()

    num_flops = []
    time_list = []

    with torch.no_grad():

      for i in range(20):

        encodings, imgs, labels = dloader[i]



def save_experiment(stats):
    """
    Saves the experiment results to a csv.
    :param config: The hyperparameters used
    :param statistics: The accuracies for the training, validation, and test sets
    """

    trial_dict = {
        'Total acc': [stats[0]],
        'Steer acc': [stats[1]],
        'Speed acc': [stats[2]]
    }

    trial_dict = pd.DataFrame(trial_dict)
    trial_dict.to_csv(os.path.join('drive', 'MyDrive', 'DriveLM', 'multi_frame_results', config.model_name, 'behavior-metrics.csv'), index=False, header=True)

# Load processors and models
model = DriveVLMT5(config)
model.to(device)

if config.lm == 'T5-Base':
    processor = T5Tokenizer.from_pretrained('google-t5/t5-base')
else:
    processor = T5Tokenizer.from_pretrained('google-t5/t5-large')

processor.add_tokens('<')

model.load_state_dict(torch.load(os.path.join('drive', 'MyDrive', 'DriveLM', 'multi_frame_results', config.model_name,
                                                          'latest_model.pth')))

# Load dataset and dataloader
test_dset = MultiFrameDataset(
    input_file=os.path.join('drive', 'MyDrive', 'DriveLM', 'data', 'multi_frame',
                            'multi_frame_test_behavior.json'),
    tokenizer=processor,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
    ])
)

# Get the loss and predictions from the model
stats = val_model(test_dset)